# Generate Acceleration Grid

The following code generates acceleartion grid used in client GPU pipeline to accelerate unit-sphere point sampling

In [1]:
import os
os.chdir(f'{os.getcwd()}/../../')
WORKING_DIR = './evaluation/uspc'

In [2]:
import pandas
import imageio
import numpy as np
import pandas as pd
from tqdm import tqdm

from utilities.geometry import fibonacci_sphere
from utilities.coordinate import spherical_to_cartesian

In [5]:
def make_acc_grid(n_anchors, v_acc_grid):
    anchors = fibonacci_sphere(n_anchors)
    
    v = np.arange(v_acc_grid)
    v = v / np.max(v) * np.pi
    u = np.arange(v_acc_grid * 2)
    u = u / np.max(u) * np.pi * 2

    uv = np.stack(np.meshgrid(u, v), axis=-1)
    uv = np.stack((uv[:, :, 1], uv[:, :, 0]), axis=-1)
    r = np.ones_like(uv).mean(axis=-1, keepdims=True)
    
    uvr = np.concatenate((uv, r), axis=-1)
    uvr_flat = uvr.reshape((-1, 3))
    uvr_cart = spherical_to_cartesian(uvr_flat)
    
    # Build cache grid
    uvr_neighbors = np.array([np.argmax(anchors @ uvr_cart[i]) for i in range(uvr_cart.shape[0])])
    acc_grid = uvr_neighbors.reshape(v_acc_grid, v_acc_grid * 2)
    
    # imageio.imsave(f'{WORKING_DIR}/figures/acc_grid_{n_anchors}_{v_acc_grid * 2}x{v_acc_grid}.png', acc_grid / (n_anchors - 1))
    
    grid_bytes = acc_grid.astype(np.uint32).tobytes()
    
    f = open(f'{WORKING_DIR}/data/grid_{n_anchors}_{v_acc_grid * 2}_{v_acc_grid}.bytes', 'wb')
    f.write(grid_bytes)
    f.close()

In [6]:
n_anchors_group = [512, 768, 1024, 1280, 1536, 1792, 2048]
v_acc_grid_group = [256, 512, 768, 1024]

In [7]:
for n_anchors in tqdm(n_anchors_group):
    for v_acc_grid in tqdm(v_acc_grid_group):
        make_acc_grid(n_anchors, v_acc_grid)

100%|██████████| 7/7 [03:56<00:00, 33.79s/it]
